In [0]:
# =========================
# (1) LIMPIEZA DE WIDGETS
# =========================
# Los widgets son parámetros editables en el notebook (referenciar a un ADLS, o el cambio de un storage account sin tocar código) 
# Permiten que el entregable sea portable y limpio, ya que los parámetros son asignados por usuarios (como Stored Procedure en SSMS),
# Y son geniales para la automatización del workflow
dbutils.widgets.removeAll()  # Elimina widgets anteriores para evitar conflictos

In [0]:
# =========================
# (2) PARÁMETROS DEL AMBIENTE
# =========================
# Aquí está el cambio: el 'container' ahora será 'dev' o 'prod'
dbutils.widgets.text("storage_account", "adlssmartdata0303ev", "ADLS Storage Account") # Nombre del Storage Account - Azure Data Lake
dbutils.widgets.text("container", "dev", "Ambiente (dev/prod)") # <--- Antes era 'raw', ahora es 'dev', desde un ambiente de desarrollo
dbutils.widgets.text("catalog_name", "catalog_footballdata", "Unity Catalog name") # Nombre del catálogo en Unity Catalog
dbutils.widgets.text("storage_credential", "credential", "UC Storage Credential") # Nombre del Storage Credential, que debe existir en el workspace

# =========================
# (3) LECTURA DE WIDGETS
# =========================
storage_account = dbutils.widgets.get("storage_account") # Captura el nombre real del Storage Account en una variable (aplica a todos)
container = dbutils.widgets.get("container") # Esto capturará 'dev'
catalog_name = dbutils.widgets.get("catalog_name") # Captura el nombre real del Catálogo en una variable
storage_credential = dbutils.widgets.get("storage_credential") # Captura el nombre real del Storage Credential en una variable

# =========================
# (4) CONSTRUCCIÓN DE URL ABFSS
# =========================
# Ahora la base es: abfss://dev@adlssmartdata0303ev.dfs.core.windows.net/
adls_base_url = f"abfss://{container}@{storage_account}.dfs.core.windows.net/" # Ruta base en ADLS Gen2 usando ABFSS
print("ADLS Base URL:", adls_base_url)
dbutils.widgets.text("adls_base_url", adls_base_url)


In [0]:
%sql
-- =========================
-- (1) BORRAR TODO EL CATÁLOGO (REINICIO LIMPIO)
-- =========================
-- Esto elimina schemas/objetos dentro del catálogo; considerar eliminarlo luego del primer run en ambiente producción?
-- Descripción: Contenedor padre de más alto nivel para organizar datos/objetos (schemas, tablas, views, volumens) ~ referenciar las notas de la clase UnitCatalog
DROP CATALOG IF EXISTS ${catalog_name} CASCADE;

-- =========================
-- (2) CREAR CATÁLOGO
-- =========================
-- Subdisivión dentro del catalogo (carpetas lógicas para tablas/volumes). Se separan por Medallion/Lakehouse
CREATE CATALOG IF NOT EXISTS ${catalog_name};

-- =========================================================
-- (3) CREAR EXTERNAL LOCATION (NUEVO ORDEN: PRIMERO ESTO)
-- =========================================================
-- !!! IMPORTANTE: Debemos crear esto ANTES que los schemas.
-- Esto le da permiso a Unity Catalog para gestionar TODO el contenedor 'dev' (o 'prod')
-- y permite que las MANAGED LOCATION de los schemas sean válidas.
CREATE EXTERNAL LOCATION IF NOT EXISTS extloc_cristal_storage
URL 'abfss://${container}@${storage_account}.dfs.core.windows.net/' --URL '${adls_base_url}'
WITH (STORAGE CREDENTIAL `${storage_credential}`);


In [0]:
%sql
-- =========================
-- (4) CREAR SCHEMAS POR CAPA (MEDALLION)
-- =========================
-- Creación de schemas pero esta ves como External Tables en vez de Managed Tables
-- Aquí es donde registramos 'raw' como una carpeta DENTRO de 'dev'
CREATE SCHEMA IF NOT EXISTS ${catalog_name}.raw 
MANAGED LOCATION 'abfss://${container}@${storage_account}.dfs.core.windows.net/'; -- MANAGED LOCATION '${adls_base_url}raw'; -- Ruta física: abfss://dev@.../raw

CREATE SCHEMA IF NOT EXISTS ${catalog_name}.bronze 
MANAGED LOCATION 'abfss://${container}@${storage_account}.dfs.core.windows.net/'; -- Ruta física: abfss://dev@.../bronze

CREATE SCHEMA IF NOT EXISTS ${catalog_name}.silver 
MANAGED LOCATION 'abfss://${container}@${storage_account}.dfs.core.windows.net/';

CREATE SCHEMA IF NOT EXISTS ${catalog_name}.golden 
MANAGED LOCATION 'abfss://${container}@${storage_account}.dfs.core.windows.net/'; -- Cambios de External location en Azure

In [0]:
%sql
-- =========================
-- (5) CREAR VOLUME (PARA DATASETS)
-- =========================
CREATE EXTERNAL VOLUME IF NOT EXISTS ${catalog_name}.raw.datasets
LOCATION 'abfss://${container}@${storage_account}.dfs.core.windows.net/raw/datasets';